### 1. Generación del Dataset Artificial
Imaginemos una tienda que vende productos a varios clientes. El dataset contendrá información sobre las siguientes variables:
- **ID del cliente**: un identificador único para cada cliente.
- **Edad del cliente**: edad del cliente en años.
- **ID del producto**: un identificador para el producto comprado.
- **Categoría del producto**: una categoría de producto (por ejemplo, tecnología, hogar, ropa).
- **Precio del producto**: precio del producto comprado.
- **Cantidad comprada**: cantidad de unidades del producto en la compra.
- **Fecha de compra**: año en el que se realizó la compra.

Para simplificar, crearemos un dataset de 100 registros simulando estas variables.

```python
import torch

# Configuración de las semillas para reproducibilidad
torch.manual_seed(0)

# Generación de datos
num_records = 100
customer_ids = torch.randint(1, 21, (num_records,))  # 20 clientes únicos
ages = torch.randint(18, 65, (num_records,))  # Edad entre 18 y 64 años
product_ids = torch.randint(1, 11, (num_records,))  # 10 productos únicos

# Categorías asignadas aleatoriamente (0: Tecnología, 1: Hogar, 2: Ropa)
categories = torch.randint(0, 3, (num_records,))

# Precios aleatorios entre 10 y 500
prices = torch.randint(10, 500, (num_records,)).float()

# Cantidad comprada entre 1 y 5 unidades
quantities = torch.randint(1, 6, (num_records,))

# Año de la compra (2018 a 2023)
purchase_years = torch.randint(2018, 2024, (num_records,))

# Creación del dataset en forma de tensor
dataset = torch.stack((customer_ids, ages, product_ids, categories, prices, quantities, purchase_years), dim=1)
print("Dataset:\n", dataset[:10])  # Mostrar las primeras 10 filas para inspección
```

Este dataset es un tensor de tamaño `[100, 7]`, donde cada columna representa una de las variables descritas.

### 2. Análisis Descriptivo del Dataset
Para este análisis, abordaremos distintos aspectos usando operaciones de PyTorch: estadísticas descriptivas, análisis de categorías, segmentación por años de compra, etc.

#### 2.1 Estadísticas Generales de la Edad y el Precio
Empezaremos con el cálculo de estadísticas descriptivas sobre las variables **Edad** y **Precio**.

```python
# Edad: columna 1, Precio: columna 4
ages_tensor = dataset[:, 1]
prices_tensor = dataset[:, 4]

# Media
age_mean = torch.mean(ages_tensor.float())
price_mean = torch.mean(prices_tensor)
print(f"Edad media: {age_mean}")
print(f"Precio medio: {price_mean}")

# Mediana
age_median = torch.median(ages_tensor)
price_median = torch.median(prices_tensor)
print(f"Edad mediana: {age_median}")
print(f"Precio mediano: {price_median}")

# Mínimo y máximo
age_min, age_max = torch.min(ages_tensor), torch.max(ages_tensor)
price_min, price_max = torch.min(prices_tensor), torch.max(prices_tensor)
print(f"Edad mínima: {age_min}, Edad máxima: {age_max}")
print(f"Precio mínimo: {price_min}, Precio máximo: {price_max}")
```

#### 2.2 Análisis por Categoría de Producto
Analicemos el promedio de precios y cantidades compradas para cada **categoría de producto**.

```python
# Filtrar por categorías
for category in range(3):
    category_mask = dataset[:, 3] == category
    category_prices = prices_tensor[category_mask]
    category_quantities = dataset[:, 5][category_mask]

    avg_price = torch.mean(category_prices)
    avg_quantity = torch.mean(category_quantities.float())
    
    print(f"\nCategoría {category} - Precio promedio: {avg_price}, Cantidad promedio: {avg_quantity}")
```

#### 2.3 Distribución de Compras por Año
Para ver cómo han variado las ventas en los últimos años, calcularemos la frecuencia de compras por cada año en el dataset.

```python
years = torch.unique(dataset[:, 6])
print("\nCompras por año:")
for year in years:
    year_count = torch.sum(dataset[:, 6] == year)
    print(f"Año {year.item()}: {year_count.item()} compras")
```

#### 2.4 Análisis del Gasto Total por Cliente
Calcularemos el **gasto total** de cada cliente, considerando el precio y la cantidad de cada compra.

```python
# Gasto total por cliente
total_spent_per_customer = torch.zeros(20)  # Asumimos que hay 20 clientes únicos con IDs de 1 a 20

for i in range(num_records):
    customer_id = dataset[i, 0] - 1  # ID del cliente ajustado para índice
    price = dataset[i, 4]
    quantity = dataset[i, 5]
    total_spent = price * quantity
    total_spent_per_customer[customer_id] += total_spent

for customer_id, spent in enumerate(total_spent_per_customer, start=1):
    print(f"Cliente {customer_id}: Gasto total = {spent.item()}")
```

#### 2.5 Segmentación de Clientes por Edad
Segmentaremos los clientes en tres grupos de edad y evaluaremos el promedio de compra para cada grupo:
- **Jóvenes**: 18 a 30 años
- **Adultos**: 31 a 50 años
- **Adultos mayores**: 51+ años

```python
young_customers = (ages_tensor >= 18) & (ages_tensor <= 30)
adult_customers = (ages_tensor >= 31) & (ages_tensor <= 50)
senior_customers = ages_tensor >= 51

print("\nPromedio de compra por grupo de edad:")
for group_name, mask in zip(["Jóvenes", "Adultos", "Adultos mayores"], [young_customers, adult_customers, senior_customers]):
    avg_purchase_amount = torch.mean((prices_tensor * dataset[:, 5])[mask])
    print(f"{group_name}: {avg_purchase_amount.item()}")
```